## Setup Only for Colab

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/hidden_mediators

In [ ]:
%ls

In [ ]:
from IPython.display import clear_output

In [ ]:
import time
!pip install -r requirements.txt
time.sleep(2)
clear_output()

In [ ]:
import time
# replace `develop` with `install` if you wont make library code changes
!python setup.py develop
time.sleep(2)
clear_output()
# Restart the session after running this

In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

# Main Logic

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import pytest
from joblib import Parallel, delayed
from sklearn.model_selection import KFold, StratifiedKFold
from proximalde.gen_data import gen_data_complex
from proximalde.proximal import proximal_direct_effect, ProximalDE, estimate_nuisances, \
    estimate_final, second_stage, _check_input, residualizeW, _gen_subsamples
from proximalde.inference import NormalInferenceResults, pvalue
from proximalde.ivreg import Regularized2SLS
from proximalde.ivtests import weakiv_tests
from proximalde.tests.utilities import gen_iv_data
from sklearn.linear_model import RidgeCV, Ridge, LinearRegression, LassoCV

#### Verifying we get the same as ivreg on the dataset in the documentation of ivreg

In [ ]:
from proximalde.tests.utilities import gen_kmenta_data
Z, X, Y, labels, controls = gen_kmenta_data()

In [ ]:
from proximalde.ivtests import weakiv_tests
weakiv_tests(Z, X, Y, controls=controls)

In [ ]:
from proximalde.diagnostics import IVDiagnostics
diag = IVDiagnostics(add_constant=True).fit(Z, X, Y)

In [ ]:
diag.influence_plot(labels=labels)
plt.show()